In [1]:
import os
import pandas as pd
import sys
sys.path.append("..")
import torch
import torch.nn as nn
import torch.optim as optim
from collections import OrderedDict
from torchvision import datasets, transforms, models
from tools.load_dataset import FoodDataset
from torch.utils.data import DataLoader
from tools.workspace_utils import active_session

Using TensorFlow backend.


In [2]:
MAX_EPOCH = 10
BATCH_SIZE = 8
LR = 0.001
log_interval = 10
val_interval = 1
N_CLASSES = 251

In [3]:
train_label_dir = os.path.join("..", "data", "train_labels.csv")
valid_label_dir = os.path.join("..", "data", "val_labels.csv")
test_label_dir = os.path.join("..", "data", "sample_submission.csv")

In [4]:
train_data_df = pd.read_csv(train_label_dir)
valid_data_df = pd.read_csv(valid_label_dir)
test_data_df = pd.read_csv(test_label_dir)

In [5]:
# data preparation
data_dir = os.path.join("D:\Rice\COMP 540", "data")
train_dir = os.path.join(data_dir, "train_set")
valid_dir = os.path.join(data_dir, "val_set")
test_dir = os.path.join(data_dir, "test_set")

norm_mean = [0.485, 0.456, 0.406]
norm_std = [0.229, 0.224, 0.225]


train_transform = transforms.Compose([
    transforms.RandomRotation(30),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(norm_mean, norm_std)
])

valid_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(norm_mean, norm_std)
])

test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(norm_mean, norm_std)
])

train_data = FoodDataset(data_dir=train_dir, data_df=train_data_df, transform=train_transform)
valid_data = FoodDataset(data_dir=valid_dir, data_df=valid_data_df, transform=valid_transform)
test_data = FoodDataset(data_dir=test_dir, data_df=test_data_df, transform=test_transform)

train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
valid_loader = DataLoader(dataset=valid_data, batch_size=BATCH_SIZE)
test_loader = DataLoader(dataset=test_data, batch_size=BATCH_SIZE)

In [6]:
# choose model
vgg_model = models.vgg16(pretrained=True)
vgg_model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [7]:
for parameter in vgg_model.parameters():
    parameter.requires_grad = False

classifier = nn.Sequential(OrderedDict({
    'fc1': nn.Linear(25088, 5000),
    'relu': nn.ReLU(),
    'drop': nn.Dropout(p=0.5),
    'fc2': nn.Linear(5000, N_CLASSES),
    'output': nn.LogSoftmax(dim=1)
}))

vgg_model.classifier = classifier

In [8]:
vgg_model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [9]:
def validation(model, validateloader, criterion):
    val_loss = 0
    accuracy = 0

    for images, labels in iter(validateloader):
        images, labels = images.to('cuda'), labels.to('cuda')

        output = model.forward(images)
        val_loss += criterion(output, labels).item()

        probabilities = torch.exp(output)

        equality = (labels.data == probabilities.max(dim=1)[1])
        accuracy += equality.type(torch.FloatTensor).mean()

    return val_loss, accuracy

In [10]:
# Loss Function
criterion = nn.NLLLoss()

In [11]:
# optimizer
optimizer = optim.Adam(vgg_model.parameters(), lr=LR)

In [12]:
os.environ['CUDA_VISIBLE_DEVICES']='0'

In [13]:
def train_classifier():
    epochs = 15
    steps = 0
    print_every = 40

    vgg_model.to('cuda')
    for e in range(epochs):

        vgg_model.train()

        running_loss = 0

        for images, labels in iter(train_loader):

            steps += 1

            images, labels = images.to('cuda'), labels.to('cuda')

            optimizer.zero_grad()

            output = vgg_model.forward(images)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            if steps % print_every == 0:
                vgg_model.eval()

                # Turn off gradients for validation, saves memory and computations
                with torch.no_grad():
                    validation_loss, accuracy = validation(vgg_model, valid_loader, criterion)

                print("Epoch: {}/{}.. ".format(e + 1, epochs),
                      "Training Loss: {:.3f}.. ".format(running_loss / print_every),
                      "Validation Loss: {:.3f}.. ".format(validation_loss / len(valid_loader)),
                      "Validation Accuracy: {:.3f}".format(accuracy / len(valid_loader)))

                running_loss = 0
                vgg_model.train()

train_classifier()

RuntimeError: CUDA out of memory. Tried to allocate 480.00 MiB (GPU 0; 3.00 GiB total capacity; 1.94 GiB already allocated; 106.05 MiB free; 1.96 GiB reserved in total by PyTorch)